In [ ]:
from torch.utils.data import Dataset, random_split
from pandas import read_csv
import seaborn as sns
import matplotlib.pyplot as plt


# dataset definition preparation 
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        self.df = read_csv(path)

        self.df.head()
        # Drop the date column
        self.df = self.df.drop('date', axis=1)
        # Drop the symbol column
        self.df = self.df.drop('symbol', axis=1)
        # store the inputs and outputs
        self.X = self.df.values[:, :-1]
        self.y = self.df.values[:, -1]
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 1))

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

    def print_info(self):
        print(self.df.info())
        print(self.df.describe())
        print(self.df.head())
        print(self.df.isnull().sum())

    def visualize(self):
        plt.figure(figsize=(10, 6))
        sns.countplot(y='open', data=self.df)
        plt.title('Spans from 2010 to the end 2016')
        plt.show()

    def check(self):
        # Step 3: Check for missing values
        print(self.df.isnull().sum())

In [ ]:
# Step 1: Load the dataset
data_set = CSVDataset('data/prices.csv')

In [ ]:
# Step 2: Summarize the dataset
data_set.print_info()

In [ ]:
# Step 3: Visualize the dataset
data_set.visualize()

In [ ]:
# Step 4: Check for missing values
data_set.check()

In [ ]:
import torch.nn as nn
import torch.nn.functional as func


# Step 5: Define the DNN architecture
class Net(nn.Module):
    def __init__(self, n_neurons=10):
        super(Net, self).__init__()
        # Define the layers of the network here
        self.fc1 = nn.Linear(5, n_neurons)
        self.fc2 = nn.Linear(n_neurons, n_neurons)
        self.fc3 = nn.Linear(n_neurons, 1)

    def forward(self, x):
        # Define the forward pass
        x = func.relu(self.fc1(x))
        x = func.relu(self.fc2(x))
        # no activation function for the la
        x = self.fc3(x)
        return x

In [ ]:
from torch.utils.data import DataLoader
from torch.optim import SGD
from torch.nn import MSELoss
from tqdm import tqdm

# Step 6: Create an instance of the network
model = Net()


# Step 7: Train the model
def train_model(train_data, training_model):
    size = len(train_data.dataset)
    # define the optimization
    criterion = MSELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    # enumerate epochs
    for epoch in tqdm(range(100), desc='Training Epochs'):
        print(f"Epoch {epoch + 1}\n-------------------------------")
        # enumerate mini batches
        for batch, (inputs, targets) in enumerate(train_data):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = training_model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

            #if batch % 100 == 0:
            loss, current = loss.item(), batch * len(inputs)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



In [ ]:
# Step 8: Prepare the data
def prepare_data(dataset):
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    return DataLoader(train, batch_size=1024, shuffle=True), DataLoader(test, batch_size=1024, shuffle=False)

In [ ]:
# Step 9: Train the model
train_dl, test_dl = prepare_data(data_set)
print(len(train_dl.dataset), len(test_dl.dataset))

train_model(train_dl, model)


In [ ]:
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetRegressor
import torch.optim as optim


# Step 10: Define a function to create a PyTorch model with variable parameters
def create_model(n_neurons=10):
    return Net(n_neurons=n_neurons)


# Step 11: Define a function to perform grid search
def perform_grid_search(X, y):
    # Define the hyperparameters
    hyperparameters = {
        'module__n_neurons': [10, 20, 30],
        'lr': [0.01, 0.001, 0.0001],
        'max_epochs': [10, 50, 100],
        'optimizer': [optim.SGD, optim.Adam],
    }

    # Create a Skorch neural network object
    net = NeuralNetRegressor(module=create_model, criterion=nn.MSELoss, iterator_train__shuffle=True)

    # Create a GridSearchCV object
    grid_search = GridSearchCV(net, hyperparameters, cv=3, scoring='neg_mean_squared_error')

    # Fit the GridSearchCV object
    grid_search.fit(X, y)

    # Print the best parameters
    print("Best parameters found: ", grid_search.best_params_)

    return grid_search.best_estimator_


# Step 12: Call the grid search function with the desired parameters
best_model = perform_grid_search(data_set.X, data_set.y)

In [ ]:
import torch

# Step 13: Initialize lists to store loss and accuracy values
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []


# Step 14: Train the model
def test_model(test_data, testing_model):
    size = len(test_data.dataset)
    # define the optimization
    criterion = MSELoss()
    current_test_loss = 0
    correct = 0

    with torch.no_grad():
        for inputs, targets in test_data:
            # compute the model output
            outputs = testing_model(inputs)
            # calculate loss
            current_test_loss += criterion(outputs, targets).item()
            # calculate accuracy
            pred = outputs.argmax(dim=1, keepdim=True)
            correct += pred.eq(targets.view_as(pred)).sum().item()

    current_test_loss /= size
    current_test_accuracy = correct / size
    print(f"Test Error: \n Accuracy: {(100 * current_test_accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return test_loss, test_accuracy


for epoch in range(100):
    # Train the model
    train_loss, train_accuracy = train_model(train_dl, model)
    # Test the model
    test_loss, test_accuracy = test_model(test_dl, model)
    # Store the loss and accuracy values
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)

# Step 15: After training, use matplotlib to plot the loss and accuracy values
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train')
plt.plot(test_losses, label='Test')
plt.title('Loss / Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train')
plt.plot(test_accuracies, label='Test')
plt.title('Accuracy / Epochs')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
class NetWithRegularization(nn.Module):
    def __init__(self, n_neurons=10, dropout_rate=0.5, *args, **kwargs):
        super(NetWithRegularization, self).__init__()
        self.fc1 = nn.Linear(5, n_neurons)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(n_neurons, n_neurons)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(n_neurons, 1)

    def forward(self, x):
        x = func.relu(self.fc1(x))
        x = self.dropout1(x)
        x = func.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x


def train_model(train_data, validation_data, training_model, l1_lambda=0.005):
    criterion = MSELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.01)  # L2 regularization
    best_val_loss = float('inf')
    for epoch in tqdm(range(100), desc='Training Epochs'):
        # Training Phase 
        training_model.train()
        for inputs, targets in train_data:
            optimizer.zero_grad()
            outputs = training_model(inputs)
            loss = criterion(outputs, targets)
            l1_norm = sum(p.abs().sum() for p in model.parameters())  # L1 regularization
            loss = loss + l1_lambda * l1_norm
            loss.backward()
            optimizer.step()

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, targets in validation_data:
                outputs = training_model(inputs)
                val_loss += criterion(outputs, targets).item()

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
        else:
            print("Early stopping")
            break

In [ ]:
# Test the original model
test_loss_original, test_accuracy_original = test_model(test_dl, model)
print(f"Original Model - Test Loss: {test_loss_original}, Test Accuracy: {test_accuracy_original}")

# Create an instance of the network with regularization
model_with_regularization = NetWithRegularization()

# Train the model with regularization
train_model(train_dl, test_dl, model_with_regularization)

# Test the model with regularization
test_loss_regularized, test_accuracy_regularized = test_model(test_dl, model_with_regularization)
print(f"Model with Regularization - Test Loss: {test_loss_regularized}, Test Accuracy: {test_accuracy_regularized}")

we can summarize the first part of the lab in those steps:  

1. A CSV dataset is loaded and preprocessed. The date and symbol columns are dropped, and the remaining data is split into inputs (X) and targets (y).

2. A neural network architecture is defined with three fully connected layers.

3. The model is trained using Stochastic Gradient Descent (SGD) and Mean Squared Error (MSE) loss.

4. A grid search is performed to find the best hyperparameters for the model.

5. The model's performance is evaluated on the test data.

6. A new model is defined with dropout layers for regularization. The training function is also modified to include L1 and L2 regularization and early stopping.

7. The regularized model is trained and evaluated on the test data, and its performance is compared with the original model.

8. The loss and accuracy of the models are plotted for each epoch.
